# Keras starter kit [full training set, UNet]

## Setup

In [61]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import albumentations as A
from albumentations.pytorch import ToTensorV2

import glob
import time
import PIL.Image as Image
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
from tqdm import tqdm
import cv2

# Data config
# DATA_DIR = '/kaggle/input/vesuvius-challenge-ink-detection/'
DATA_DIR = '/home/fummicc1/codes/competitions/kaggle-ink-detection'
BUFFER = 128  # Half-size of papyrus patches we'll use as model inputs
Z_LIST = list(range(0, 65, 3))  # Offset of slices in the z direction
Z_DIM = len(Z_LIST)  # Number of slices in the z direction. Max value is 64 - Z_START
SHARED_HEIGHT = 4000  # Height to resize all papyrii

# (y, x)
val_location = (600, 500)
val_zone_size = (1000, 2000)

# Model config
BATCH_SIZE = 128
USE_MIXED_PRECISION = False
USE_JIT_COMPILE = False

device = torch.device("cuda")
threshold = 0.35
num_workers = 4
exp = 1e-7

In [62]:
from scipy.stats import median_abs_deviation

def calculate_MAD(volume):
    all_MAD = median_abs_deviation(volume, axis=[0, 1])
    return all_MAD
    
def calculate_median(volume):
    all_median = np.median(volume, axis=[0, 1])
    return all_median

In [63]:
all_median = np.array([19581., 19608., 19635., 19654., 19710., 19863., 20186., 20833.,
        21908., 22622., 21038., 18247., 17848., 19475., 21128., 22109.,
        22629., 22924., 23099., 23198., 23261., 23297.])

In [64]:
all_MAD = np.array([12424., 12524., 12643., 12756., 12864., 12935., 13006., 13152.,
        13550., 14332., 14870., 12094.,  7500.,  5350.,  4056.,  3408.,
         3067.,  2859.,  2725.,  2643.,  2588.,  2557.])

In [65]:
possible_max_input = ((2 ** 16 - 1) / all_median.min())
possible_max_input

3.67183998207082

In [66]:
def resize(img):
    current_height, current_width = img.shape    
    aspect_ratio = current_width / current_height
    new_width = int(SHARED_HEIGHT * aspect_ratio)
    new_size = (new_width, SHARED_HEIGHT)
    # (W, H)の順で渡すが結果は(H, W)になっている
    img = cv2.resize(img, new_size)
    return img

def load_mask(split, index):
    img = cv2.imread(f"{DATA_DIR}/{split}/{index}/mask.png", 0) // 255
    img = resize(img)    
    return img


def load_labels(split, index):
    img = cv2.imread(f"{DATA_DIR}/{split}/{index}/inklabels.png", 0) // 255
    img = resize(img)
    return img


In [67]:
def load_volume(split, index):
    # Load the 3d x-ray scan, one slice at a time
    all = sorted(glob.glob(f"{DATA_DIR}/{split}/{index}/surface_volume/*.tif"))
    z_slices_fnames = [all[i] for i in range(len(all)) if i in Z_LIST]
    assert len(z_slices_fnames) == Z_DIM
    z_slices = []
    for z, filename in  tqdm(enumerate(z_slices_fnames)):
        img = cv2.imread(filename, -1)
        img = resize(img)
        z_slices.append(img)
    return np.stack(z_slices, axis=-1)

In [68]:
def sample_random_location(shape):
    random_train_x = np.random.randint(low=BUFFER, high=shape[1] - BUFFER - 1, size=())
    random_train_y = np.random.randint(low=BUFFER, high=shape[0] - BUFFER - 1, size=())
    random_train_location = np.stack([random_train_y, random_train_x], axis=-1)
    return random_train_location


def is_in_masked_zone(location, mask):
    return mask[location[0], location[1]] > 0

def is_in_val_zone(location, val_location, val_zone_size):
    x = location[1]
    y = location[0]
    x_match = val_location[1] - BUFFER <= x <= val_location[1] + val_zone_size[1] + BUFFER
    y_match = val_location[0] - BUFFER <= y <= val_location[0] + val_zone_size[0] + BUFFER
    return x_match and y_match


In [69]:
printed = False

def extract_subvolume(location, volume):
    global printed
    # print(np.unique(volume, return_counts=True, return_index=True))
    x = location[0]
    y = location[1]
    subvolume = volume[x-BUFFER:x+BUFFER, y-BUFFER:y+BUFFER, :].astype(np.float32)
    # print("subvolume[:, :, 0]", subvolume[:, :, 0])
    median = np.full_like(subvolume, all_median).astype(np.float32)
    MAD = np.full_like(subvolume, all_MAD).astype(np.float32)
    # mean = np.mean(subvolume, axis=2)
    # mean = np.stack([mean for i in range(Z_DIM)], axis=2) + exp
    # MAD = median_abs_deviation(subvolume, axis=2)
    # print("MAD", MAD[0, 0, :])
    # print("mean", mean)
    # print("median", median[0, 0, :])
    
    subvolume = (subvolume / median)
    
    if not printed:
        print("subvolume after taking care of median and MAD", subvolume)
        printed = True
    
    return subvolume

In [70]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import OneHotEncoder

class SubvolumeDataset(Dataset):
    def __init__(self, locations, volume, labels, buffer, is_train: bool, return_location: bool = False):
        self.locations = locations
        self.volume = volume
        self.labels = labels        
        self.buffer = buffer
        self.is_train = is_train
        self.return_location = return_location

    def __len__(self):
        return len(self.locations)

    def __getitem__(self, idx):
        label = None
        location = np.array(self.locations[idx])
        y, x = location[0], location[1]

        subvolume = extract_subvolume(location, self.volume)        
        # print("subvolume", subvolume)
        # print("labels", labels)
        # subvolume = subvolume.numpy()
        subvolume = subvolume
        
        if self.labels is not None:
            label = self.labels[y - self.buffer:y + self.buffer, x - self.buffer:x + self.buffer]
            # print("label", label)
            # n_category = 2
            # label = np.eye(n_category)[label]
            label = np.stack([label], axis=-1)
            # label = label.numpy()
            # print("label.shape", label.shape
        
        if self.is_train and label is not None:            
            
            # print("label", label.dtype)
            # print("subvolume in dataset (before aug)", subvolume)    
            size = int(BUFFER * 1.5)
            performed = A.Compose([            
                A.ToFloat(max_value=possible_max_input),
                A.HorizontalFlip(p=0.5), # 水平方向に反転
                A.VerticalFlip(p=0.5), # 水平方向に反転
                A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0), # シフト、スケーリング、回転
                A.PadIfNeeded(min_height=size, min_width=size, always_apply=True, border_mode=0), # 必要に応じてパディングを追加
                A.RandomCrop(height=size, width=size, always_apply=True), # ランダムにクロップ, Moduleの中で計算する際に次元がバッチ内で揃っている必要があるので最後にサイズは揃える
                # A.GaussNoise(p=0.2), # ガウスノイズを追加　Note: これは背景とinkの境界を曖昧にしてしまうため不適切かもしれない
                A.Perspective(p=0.5), # パースペクティブ変換                   
                # A.OneOf(
                #     [
                #         A.Sharpen(p=1),
                #         A.Blur(blur_limit=3, p=1),
                #         A.MotionBlur(blur_limit=3, p=1),
                #     ],
                #     p=0.9,
                # ),
                A.Resize(BUFFER * 2, BUFFER * 2, always_apply=True),
                A.FromFloat(max_value=possible_max_input),
            ])(image=subvolume, mask=label)            
            subvolume = performed["image"]            
            label = performed["mask"]
            # print("subvolume in dataset (after aug)", subvolume)
            # print("label", label.dtype)
            # print("subvolume", subvolume.dtype)
            # →C, H, W
            subvolume = torch.from_numpy(subvolume.transpose(2, 0, 1).astype(np.float64))
            # print(performed)
            # print(subvolume.shape, label.shape)
            # H, W, C → C, H, W
            label = torch.from_numpy(label.transpose(2, 0, 1).astype(np.uint8)) 
        else:
            performed = A.Compose([  
                A.ToFloat(max_value=possible_max_input),                
                # A.Normalize(
                #     mean=[mean],
                #     std=[std],
                # ),
                A.FromFloat(max_value=possible_max_input),
            ])(image=subvolume)
            subvolume = performed["image"]
            subvolume = torch.from_numpy(subvolume.transpose(2, 0, 1).astype(np.float64))
            if label is not None:
                label = torch.from_numpy(label.transpose(2, 0, 1).astype(np.uint8)) 
        if self.return_location:
            return subvolume, location
        return subvolume, label        

In [71]:
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-5):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        preds = preds.view(-1)  # Flatten the tensor
        targets = targets.view(-1)  # Flatten the tensor

        intersection = (preds * targets).sum()
        union = preds.sum() + targets.sum()

        dice = (2. * intersection + self.smooth) / (union + self.smooth)

        return 1 - dice

In [72]:
class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.5, beta=0.5, smooth=1e-5):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.dice_loss = DiceLoss(smooth)
        self.bce_loss = nn.BCELoss()

    def forward(self, preds, targets):
        dice_loss = self.dice_loss(preds, targets)
        bce_loss = self.bce_loss(preds, targets)
        return self.alpha * dice_loss + self.beta * bce_loss


In [73]:

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()

        def conv_block(in_channels, out_channels):
            return nn.Sequential(                
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
            )

        def transpose_conv_block(in_channels, out_channels):
            return nn.Sequential(                
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
            )
            
        self.first_conv = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )

        self.encoder = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(2**(i + 5), 2**(i + 6), kernel_size=3, stride=2, padding=1),
                nn.BatchNorm2d(2**(i + 6)),
                nn.ReLU(),
                nn.Conv2d(2**(i + 6), 2**(i + 6), kernel_size=3, padding=1),
                nn.BatchNorm2d(2**(i + 6)),
                nn.ReLU(),
            )
            for i in range(1, 4)
        ])


        self.middle = nn.Sequential(
            conv_block(512, 1024),
            conv_block(1024, 512),
        )
        
        self.decoder = nn.ModuleList([
            nn.Sequential(
                transpose_conv_block(2 ** (i + 7), 2 ** (i + 6)),
                transpose_conv_block(2 ** (i + 6), 2 ** (i + 5)),
                nn.Upsample(scale_factor=2, mode="nearest"),
            )
            for i in range(3, 0, -1)
        ])
        self.final_decoder = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            # nn.BatchNorm2d(64),
            # nn.ReLU(),
            nn.Conv2d(64, out_channels, kernel_size=3, padding=1),
            # nn.BatchNorm2d(out_channels),
        )
        self.activation = nn.Sigmoid()

    def forward(self, x):
        # print("input:", x)
        skip_connections = []
        x = self.first_conv(x)
        skip_connections.append(x)
        for layer in self.encoder:
            x = layer(x)
            skip_connections.append(x)
        # print("encoder ok", x.shape)

        x = self.middle(x)
        
        # print("middle ok", x.shape)
        for i, layer in enumerate(self.decoder):                        
            skips = skip_connections.pop()
            # print(f"decoder before skip connection: {i}: ok", x.shape, skips.shape)
            x = torch.cat([x, skips], dim=1)  # Concatenate along channel dimension
            # print(f"decoder with skip connection {i}: ok", x.shape)            
            x = layer(x)
            # print(f"decoder {i}: ok", x.shape)
        # print("decoder ok")
        x = torch.cat([x, skip_connections[0]], dim=1)
        x = self.final_decoder(x)
        x = self.activation(x)
        # print("final out", x)
        return x

In [74]:
device = torch.device("cuda")

In [75]:
model = UNet(Z_DIM, 1)
model = nn.DataParallel(model)
# model.load_state_dict(torch.load(f"/kaggle/input/ink-detection/model.pt"))
model.load_state_dict(torch.load("model.pt"))
model = model.to(device)

## Load up the training data

In [76]:

import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

def compute_predictions_map(split, index):
    
    print(f"Load data for {split}/{index}")

    test_volume = load_volume(split=split, index=index)
    test_mask = load_mask(split=split, index=index)    

    test_locations = []
    stride = BUFFER // 2
    for y in range(BUFFER, test_volume.shape[0] - BUFFER, stride):
        for x in range(BUFFER, test_volume.shape[1] - BUFFER, stride):
            test_locations.append((y, x))

    print(f"{len(test_locations)} test locations (before filtering by mask)")

    # filter locations inside the mask
    test_locations = [loc for loc in test_locations if is_in_masked_zone(loc, test_mask)]
    
    print(f"{len(test_locations)} test locations (after filtering by mask)")

    test_ds = SubvolumeDataset(test_locations, test_volume, None, BUFFER, is_train=False, return_location=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, num_workers=num_workers)

    # shape: (X, Y, C)
    predictions_map = np.zeros_like(test_volume[:, :, 0]).transpose((1, 0))[:, :, np.newaxis].astype(np.float64)
    
    print("test_volume.shape", test_volume.shape)
    print("predictions_map.shape", predictions_map.shape)

    print(f"Compute predictions")

    model.eval()  # set model to evaluation mode
    with torch.no_grad():    
        for patch_batch, loc_batch in tqdm(test_loader):
            loc_batch = loc_batch.to(device).long()
            patch_batch = patch_batch.to(device).float()
            predictions = model(patch_batch)
            # print("predictions", predictions)
            # print("predictions", predictions[:, :, 0, 0])
            # print("Softmaxed predictions where conf is gt threshold", predictions[predictions.gt(threshold)])
            # →(BATCH, W, H, C)
            predictions = torch.permute(predictions, (0, 3, 2, 1))
            predictions = predictions.cpu().numpy()  # move predictions to cpu and convert to numpy
            for (y, x), pred in zip(loc_batch, predictions):
                # print("index: ", index ,"x, y, pred", x.item(), y.item(), pred[BUFFER, BUFFER, :].item(), file=open('log.out', 'a'))
                predictions_map[
                    x - BUFFER : x + BUFFER, y - BUFFER : y + BUFFER, :
                ][pred > threshold] = 1
    print("predictions_map", predictions_map, file=open("predictions_map", "w"))
    return predictions_map


In [77]:
from skimage.transform import resize as resize_ski
import pathlib

In [78]:
def rle(predictions_map, threshold):
    flat_img = (np.where(predictions_map.flatten() >= threshold, 1, 0)).astype(np.uint8)
    
    # Add padding at the beginning and end
    flat_img = np.pad(flat_img, pad_width=1, mode='constant', constant_values=0)

    starts = np.where((flat_img[:-1] == 0) & (flat_img[1:] == 1))[0]
    ends = np.where((flat_img[:-1] == 1) & (flat_img[1:] == 0))[0]

    lengths = ends - starts

    return " ".join(map(str, np.c_[starts, lengths].flatten()))


In [79]:
def update_submission(predictions_map, index):
    rle_ = rle(predictions_map, threshold=threshold)
    print(f"{index}," + rle_, file=open('submission.csv', 'a'))

In [80]:
print("Id,Predicted", file=open('submission.csv', 'w'))
kind = "test"
folder = pathlib.Path(DATA_DIR) / kind
for p in list(folder.iterdir()):
    index = p.stem
    predictions_map = compute_predictions_map(split=kind, index=index)
    original_size = cv2.imread(DATA_DIR + f"/{kind}/{index}/mask.png", 0).shape[:2]
    # W, H, C → H, W, C
    predictions_map = predictions_map.transpose((1, 0, 2))    
    predictions_map = resize_ski(predictions_map, (original_size[0], original_size[1], 1)).squeeze(axis=-1)    
    print("original predictions_map size", predictions_map.shape)    
    # H, W → W, H
    update_submission(predictions_map, index)
    plt.imsave(f"{index}.png", predictions_map, cmap="gray")

Load data for test/a


22it [00:00, 26.80it/s]


8378 test locations (before filtering by mask)
5493 test locations (after filtering by mask)
test_volume.shape (4000, 9284, 22)
predictions_map.shape (9284, 4000, 1)
Compute predictions


  0%|          | 0/43 [00:00<?, ?it/s]

subvolume after taking care of median and MADsubvolume after taking care of median and MADsubvolume after taking care of median and MAD   [[[1.181809   1.0858324  0.90557677 ... 0.9121476  0.9441125  1.0183715 ]
  [1.2042286  1.0733885  0.8417622  ... 0.90947497 0.9470788  1.020861  ]
  [1.2228692  1.0633925  0.7840591  ... 0.911113   0.94694984 1.0227067 ]
  ...
  [1.0962667  0.94481844 1.0101858  ... 0.9485732  0.9823309  0.96716315]
  [1.0885552  0.94135046 0.99954164 ... 0.94930595 0.9904131  0.9683221 ]
  [1.0857464  0.9508364  0.9867074  ... 0.95163375 0.9990112  0.97433144]]

 [[1.1794087  1.0708895  0.8061115  ... 0.91253555 0.9475087  1.0339528 ]
  [1.196466   1.0572726  0.76032597 ... 0.91658765 0.9555479  1.0352836 ]
  [1.2084674  1.0483986  0.7233512  ... 0.9223209  0.96264136 1.0358845 ]
  ...
  [1.0929472  0.9233986  0.9733639  ... 0.97068715 0.98095524 0.9769498 ]
  [1.0842143  0.92166466 0.95788133 ... 0.97374773 0.98929536 0.97523284]
  [1.0830396  0.9405855  0.9439266

...

4071 test locations (before filtering by mask)
2322 test locations (after filtering by mask)
test_volume.shape (4000, 4642, 22)
predictions_map.shape (4642, 4000, 1)
Compute predictions


  0%|          | 0/19 [00:00<?, ?it/s]

subvolume after taking care of median and MADsubvolume after taking care of median and MAD  subvolume after taking care of median and MAD [[[0.50472397 0.21710527 1.69315    ... 1.0144409  0.9963458  1.0039061 ]
  [0.6210102  0.1384639  1.1390884  ... 1.0155185  0.9912729  1.0094433 ]
  [0.609213   0.13958588 0.58538324 ... 1.0309079  0.9874468  1.0082843 ]
  ...
  [2.2488127  0.8003366  1.3144895  ... 1.064359   1.0214522  1.051423  ]
  [2.0783923  0.5498266  1.2754774  ... 1.0897491  1.0214952  1.0448127 ]
  [1.8832542  0.39545083 1.3606825  ... 1.1265196  1.0235587  1.0463579 ]]

 [[0.6013993  0.26065892 1.3878788  ... 1.0312096  0.9996131  0.9642443 ]
  [0.53541696 0.1874235  0.90756303 ... 1.0119407  0.98783374 0.97304374]
  [0.51018846 0.15080579 0.43366438 ... 0.9959048  0.9663385  0.9783234 ]
  ...
  [2.4907308  0.9226846  1.2567863  ... 1.0572032  1.0444092  1.0486758 ]
  [2.3992646  0.7655039  1.2501655  ... 1.0712562  1.0520184  1.0323647 ]
  [2.2255247  0.6563138  1.3172396

100%|██████████| 19/19 [00:08<00:00,  2.12it/s]


original predictions_map size (5454, 6330)
